# LAB 2 - LINEAR REGRESSION IN PYTHON

This lab among other things introduces a Linear Regression package from the `statsmodels` library.



The `statsmodels` library includes a variety of functions that are helpful for data exploration and statiscal models, its documentation can be found here: https://www.statsmodels.org/stable/index.html

We will see two ways of using Multiple Linear Regression:

- 1) MLR with numerical variables
- 2) MLR with numerical + categorical variables

More specifically, we will use Linear Regression to predict the quality of the wines as measured by their 'Auction Index'




In [82]:
import numpy as np
import pandas as pd

As in the previous Lab, we use pandas to read the data from a `.csv` file, and consolidate it into a `Dataframe` object.

In [83]:
wine = pd.read_csv('wine_agg.csv')
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 9 columns):
Year               46 non-null int64
LogAuctionIndex    46 non-null float64
WinterRain         46 non-null float64
HarvestRain        46 non-null float64
GrowTemp           46 non-null float64
HarvestTemp        46 non-null float64
Age                46 non-null int64
FrancePop          46 non-null float64
USAlcConsump       46 non-null float64
dtypes: float64(7), int64(2)
memory usage: 3.4 KB


In [84]:
wine.head(5)

,Year,LogAuctionIndex,WinterRain,HarvestRain,GrowTemp,HarvestTemp,Age,FrancePop,USAlcConsump
0,1952,7.4950,566.4,165.5,17.28,14.39,63,42.46,7.85
1,1953,8.0393,653.3,75.6,16.94,17.64,62,42.75,8.03
2,1955,7.6858,504.3,129.5,17.30,17.13,60,43.43,7.84
3,1957,6.9845,390.8,110.4,16.31,16.47,58,44.31,7.77
4,1958,6.7772,538.8,187.0,16.82,19.72,57,44.79,7.74


In [85]:
# # Plot scatter matrix
# # density plots on the diagonal and correlation printed in the upper triangle
# ggscatmat(wine, columns = 2:9, alpha = 0.8)

In [86]:
wine.corr()

,Year,LogAuctionIndex,WinterRain,HarvestRain,GrowTemp,HarvestTemp,Age,FrancePop,USAlcConsump
Year,1.000000,-0.011346,-0.032424,0.128569,0.600924,0.277000,-1.000000,0.986137,0.131949
LogAuctionIndex,-0.011346,1.000000,0.058326,-0.525882,0.559727,0.469832,0.011346,-0.076993,-0.271188
WinterRain,-0.032424,0.058326,1.000000,-0.120258,-0.214181,-0.046874,0.032424,-0.045196,0.004174
HarvestRain,0.128569,-0.525882,-0.120258,1.000000,0.036437,-0.410439,-0.128569,0.112394,-0.220185
GrowTemp,0.600924,0.559727,-0.214181,0.036437,1.000000,0.513076,-0.600924,0.516918,-0.352116
HarvestTemp,0.277000,0.469832,-0.046874,-0.410439,0.513076,1.000000,-0.277000,0.250442,-0.035964
Age,-1.000000,0.011346,0.032424,-0.128569,-0.600924,-0.277000,1.000000,-0.986137,-0.131949
FrancePop,0.986137,-0.076993,-0.045196,0.112394,0.516918,0.250442,-0.986137,1.000000,0.269647
USAlcConsump,0.131949,-0.271188,0.004174,-0.220185,-0.352116,-0.035964,-0.131949,0.269647,1.000000


# 1) MLR with numerical variables

## Dataset train-test splitting

Next, we will split the dataset into a training set and a test set. There are various ways of splitting the dataset, we will first do an example of chronological separation. Eventually we'll cover randomized splitting as well.

We can do chronological splitting sing Boolean predicates, the same way we used them in the last lab:
- A & B
- A | B
- A == B
- A <= B
- A > B

 

In [87]:
wine_train = wine[wine['Year'] <= 1985]
wine_test = wine[wine['Year'] > 1985]

len(wine_train), len(wine_test)

(31, 15)

In [88]:
### produce wine_train2 which is composed of wine that's older than 40
wine_train2 = wine[wine['Age'] > 40]

### How many rows does this have? (use the function nrows(dataframe)) 
len(wine_train2)

20

In [89]:
## produce wine.train3 which is composed of wine less than Year 1985 or older than year 1990. 
wine_train3 = wine[(wine['Year'] <= 1985) | (wine['Year'] > 1990)]

# How many rows is the rest of the dataset, other than wine.train3? 
len(wine_train3)

41

## Training the model

We will show two ways of passing the data into the model:

- We can select the columns of interest that will constitute our matrices

- Or we can use syntaxis that follows R-style formulas




### Training the model (matrix style)

https://www.statsmodels.org/stable/examples/notebooks/generated/ols.html

In [90]:
# import the library that contains all the functions/modules related to the regression model
import statsmodels.api as sm

# choose the features to be used
cols = ['WinterRain', 'HarvestRain', 'GrowTemp', 'HarvestTemp', 'Age', 'FrancePop', 'USAlcConsump']
X_train = wine_train[cols]
y_train = wine_train['LogAuctionIndex']

# add an intercept as the standard model doesn't automatically fit one
X_train = sm.add_constant(X_train)

# fit the data to the model
model1 = sm.OLS(y_train, X_train).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:        LogAuctionIndex   R-squared:                       0.789
Model:                            OLS   Adj. R-squared:                  0.725
Method:                 Least Squares   F-statistic:                     12.31
Date:                Sun, 11 Oct 2020   Prob (F-statistic):           1.86e-06
Time:                        17:16:19   Log-Likelihood:                -5.0600
No. Observations:                  31   AIC:                             26.12
Df Residuals:                      23   BIC:                             37.59
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -4.9663      9.382     -0.529   

In [91]:
# # # Confidence interval plot
# # ggcoef(
# #   mod1,
# #   vline_color = "red",
# #   vline_linetype =  "solid",
# #   errorbar_color = "blue",
# #   errorbar_height = .25,
# #   exclude_intercept = TRUE
# # )

# # https://www.statsmodels.org/stable/graphics.html
    
# from matplotlib import pyplot as plt
# fig = sm.qqplot(model1.resid, fit=True, line="45")
# plt.show()

# from statsmodels.graphics.plot_grids import scatter_ellipse
# fig = plt.figure()
# scatter_ellipse(wine_train, varnames=wine_train.columns, fig=fig)
# plt.show()

### Training the model (using R-style formulas)

https://www.statsmodels.org/stable/example_formulas.html#categorical-variables

One of the main advantages of using this type of notation is the fact the categorical variables are handled automatically. 

Furthermore, the constant representing the intercept is generated by default in `smf.ols`.

For the remained of this lab, we will stick with the formula style notation as presented below:

In [92]:
import statsmodels.formula.api as smf

model1 = smf.ols(formula='LogAuctionIndex ~ WinterRain + HarvestRain + GrowTemp + HarvestTemp + Age + FrancePop + USAlcConsump',
                 data=wine_train).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:        LogAuctionIndex   R-squared:                       0.789
Model:                            OLS   Adj. R-squared:                  0.725
Method:                 Least Squares   F-statistic:                     12.31
Date:                Sun, 11 Oct 2020   Prob (F-statistic):           1.86e-06
Time:                        17:16:19   Log-Likelihood:                -5.0600
No. Observations:                  31   AIC:                             26.12
Df Residuals:                      23   BIC:                             37.59
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -4.9663      9.382     -0.529   

### Evaluating the model

If we want to use evaluation metrics that are not contained in the standard package, such as Out-of-sample R^2, we can manipulate our dataframe's columns and take advantage of `numpy's` matrix operation functions

In [93]:
# compute out-of-sample R squared using the test set
X_test = wine_test[cols]
y_test = wine_test['LogAuctionIndex']
X_test = sm.add_constant(X_test)

y_pred = model1.predict(X_test)
SSE = np.sum((y_test - y_pred)**2)
SST = np.sum((y_test - np.mean(y_train))**2)
OSR2 = 1 - SSE/SST
print(OSR2)

0.5377506779454809


/Users/stefanozavagli/anaconda3/envs/ee127/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<font color='blue'>
    
### [Sidenote - Programming Exploration]

Generally speaking, there are two types of programming exploration: 

1. Finding which function to utilize
2. Understanding how to use a function once you've found it 

For 1   -- Try to target the problem you want to address as succinctly as possible in google.

---------- Read the description of the function and what it returns

For 2   -- What to do with new functions that you've never seen before? 

---------- Read the arguments carefully, look at examples, try it out and test output

</font>

### Variance Inflation Factor (VIF)

In [53]:
# calculate Variance Inflation Factor for each explanatory variable
from statsmodels.stats.outliers_influence import variance_inflation_factor

cols = ['WinterRain', 'HarvestRain', 'GrowTemp', 'HarvestTemp', 'Age', 'FrancePop', 'USAlcConsump']
values = wine_train[cols].values
num_vars = wine_train[cols].shape[1]

vif = [variance_inflation_factor(values, i) for i in range(num_vars)] 
print('\nVIF:')
print(pd.Series(np.array(vif), index=cols))


VIF:
WinterRain        25.652357
HarvestRain        7.700264
GrowTemp         937.963661
HarvestTemp      317.345758
Age               70.892270
FrancePop       1490.840277
USAlcConsump     697.325915
dtype: float64


### A better model...

In [55]:
# remove FrancePop
model2 = smf.ols(formula='LogAuctionIndex ~ WinterRain + HarvestRain + GrowTemp + HarvestTemp + Age + USAlcConsump',
                 data=wine_train).fit()
print(model2.summary())

cols = ['WinterRain', 'HarvestRain', 'GrowTemp', 'HarvestTemp', 'Age', 'USAlcConsump']
values = wine_train[cols].values
num_vars = wine_train[cols].shape[1]

vif = [variance_inflation_factor(values, i) for i in range(num_vars)] 
print('\nVIF:')
print(pd.Series(np.array(vif), index=cols))

                            OLS Regression Results                            
Dep. Variable:        LogAuctionIndex   R-squared:                       0.789
Model:                            OLS   Adj. R-squared:                  0.736
Method:                 Least Squares   F-statistic:                     14.95
Date:                Sun, 11 Oct 2020   Prob (F-statistic):           4.60e-07
Time:                        15:30:32   Log-Likelihood:                -5.0902
No. Observations:                  31   AIC:                             24.18
Df Residuals:                      24   BIC:                             34.22
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -6.8405      3.071     -2.228   

In [58]:
# remove USAlcConsump
model3 = smf.ols(formula='LogAuctionIndex ~ WinterRain + HarvestRain + GrowTemp + HarvestTemp + Age',
                 data=wine_train).fit()
print(model3.summary())

cols = ['WinterRain', 'HarvestRain', 'GrowTemp', 'HarvestTemp', 'Age']
values = wine_train[cols].values
num_vars = wine_train[cols].shape[1]

vif = [variance_inflation_factor(values, i) for i in range(num_vars)] 
print('\nVIF:')
print(pd.Series(np.array(vif), index=cols))

                            OLS Regression Results                            
Dep. Variable:        LogAuctionIndex   R-squared:                       0.785
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     18.30
Date:                Sun, 11 Oct 2020   Prob (F-statistic):           1.21e-07
Time:                        15:32:25   Log-Likelihood:                -5.3480
No. Observations:                  31   AIC:                             22.70
Df Residuals:                      25   BIC:                             31.30
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -5.2152      1.672     -3.119      

In [59]:
# remove HarvestTemp
model4 = smf.ols(formula='LogAuctionIndex ~ WinterRain + HarvestRain + GrowTemp + Age',
                 data=wine_train).fit()
print(model4.summary())

cols = ['WinterRain', 'HarvestRain', 'GrowTemp', 'Age']
values = wine_train[cols].values
num_vars = wine_train[cols].shape[1]

vif = [variance_inflation_factor(values, i) for i in range(num_vars)] 
print('\nVIF:')
print(pd.Series(np.array(vif), index=cols))

                            OLS Regression Results                            
Dep. Variable:        LogAuctionIndex   R-squared:                       0.785
Model:                            OLS   Adj. R-squared:                  0.752
Method:                 Least Squares   F-statistic:                     23.78
Date:                Sun, 11 Oct 2020   Prob (F-statistic):           2.31e-08
Time:                        15:32:38   Log-Likelihood:                -5.3569
No. Observations:                  31   AIC:                             20.71
Df Residuals:                      26   BIC:                             27.88
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -5.2164      1.640     -3.180      

In [ ]:
## produce a model that outputs the effect of an increase of log(1) of USAlcConsump controlling for Age
# whats' the coefficient associated with 



### Feature Engineering

In [60]:
# let's create a new variable and test its predictive power
model5 = smf.ols(formula='LogAuctionIndex ~ WinterRain + HarvestRain + GrowTemp*Age',
                 data=wine_train).fit()
print(model5.summary())

                            OLS Regression Results                            
Dep. Variable:        LogAuctionIndex   R-squared:                       0.786
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     18.34
Date:                Sun, 11 Oct 2020   Prob (F-statistic):           1.19e-07
Time:                        15:34:42   Log-Likelihood:                -5.3227
No. Observations:                  31   AIC:                             22.65
Df Residuals:                      25   BIC:                             31.25
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -7.3978      9.432     -0.784   

## 2) MLR with numerical + categorical variables

For this part, we will use a second dataset, `wine_disagg.csv`, which contains additional information related to the Wineries. The `Winery` variable is a `string object`, but we can do some transformations that will help us fit it into the continuous model

In [62]:
wine_new = pd.read_csv("wine_disagg.csv")
wine_new_train = wine_new[wine_new['Year'] <= 1985]
wine_new_test = wine_new[wine_new['Year'] > 1985]

wine_new.info()
wine_new.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 10 columns):
Winery          147 non-null object
Year            147 non-null int64
Age             147 non-null int64
LogAuction      147 non-null float64
WinterRain      147 non-null float64
HarvestRain     147 non-null float64
GrowTemp        147 non-null float64
HarvestTemp     147 non-null float64
FrancePop       147 non-null float64
USAlcConsump    147 non-null float64
dtypes: float64(7), int64(2), object(1)
memory usage: 11.6+ KB


,Winery,Year,Age,LogAuction,WinterRain,HarvestRain,GrowTemp,HarvestTemp,FrancePop,USAlcConsump
0,Cheval Blanc,1952,63,6.653108,566.4,165.5,17.28,14.39,42.46,7.85
1,Lafite-Rothschild,1952,63,6.861502,566.4,165.5,17.28,14.39,42.46,7.85
2,Cheval Blanc,1953,62,6.664192,653.3,75.6,16.94,17.64,42.75,8.03
3,Cheval Blanc,1955,60,6.311426,504.3,129.5,17.30,17.13,43.43,7.84
4,Lafite-Rothschild,1955,60,6.550209,504.3,129.5,17.30,17.13,43.43,7.84


In [64]:
# Simple regression using new data, not yet incorporating Winery variable
modOld = smf.ols(formula='LogAuction ~ WinterRain + HarvestRain + GrowTemp + Age',
                 data=wine_new_train).fit()
print(modOld.summary())

                            OLS Regression Results                            
Dep. Variable:             LogAuction   R-squared:                       0.222
Model:                            OLS   Adj. R-squared:                  0.182
Method:                 Least Squares   F-statistic:                     5.567
Date:                Sun, 11 Oct 2020   Prob (F-statistic):           0.000539
Time:                        15:38:17   Log-Likelihood:                -112.14
No. Observations:                  83   AIC:                             234.3
Df Residuals:                      78   BIC:                             246.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.2184      3.431     -0.064      

In [ ]:
# # Plot data
# # ggplot(), geom_line() are from package "ggplot2"
# # see http://ggplot2.org for reference 
# # ggplot(data, aes(x = x_data, y = y_data))
# # +geom_line() make line plots
# # +geom_point() make point plots
# # xlab(labelx),ylab(labely) axis label
# # ggtitle(title)

# ggplot(wine.new, aes(x=Year, y=LogAuction, color=Winery)) + geom_line() +
#   xlab("Vintage Year") + ylab("log(2015 Auction Price)")

# # linting: each function argument gets its own line. 
#   # each addition gets its own line


# # Your turn: plot same data, make point plot,  name the axis and title.
# ggplot(wine.new, aes(x=Year, y=LogAuction, color=Winery)) + geom_point() +
#   xlab("Vintage Year") + ylab("log(2015 Auction Price)")+ggtitle("MyTitle")

### Two Wineries
Before constructing a complete model for all the wineries, let's first attempt to regress on only 2 wineries. We pick Cheval Blanc and Cos d'Estournel as an example

In [72]:
wine_two = wine_new[(wine_new['Winery'] == 'Cheval Blanc') | (wine_new['Winery'] == 'Cos d\'Estournel')]

# ggplot(wine.two, aes(x=Year, y=LogAuction, color=Winery)) + geom_line() +
#   xlab("Vintage Year") + ylab("log(2015 Auction Price)")

wine_two_train = wine_two[wine_two['Year'] <= 1985]
wine_two_test = wine_two[wine_two['Year'] > 1985]
wine_two_train.tail()

,Winery,Year,Age,LogAuction,WinterRain,HarvestRain,GrowTemp,HarvestTemp,FrancePop,USAlcConsump
73,Cos d'Estournel,1983,32,4.558498,698.3,119.3,17.87,18.57,54.77,10.46
76,Cheval Blanc,1984,31,5.558641,572.6,144.8,16.71,16.24,55.03,10.22
77,Cos d'Estournel,1984,31,3.426865,572.6,144.8,16.71,16.24,55.03,10.22
80,Cheval Blanc,1985,30,6.018934,667.1,37.2,17.19,19.56,55.28,9.88
81,Cos d'Estournel,1985,30,4.885072,667.1,37.2,17.19,19.56,55.28,9.88


### Passing a categorical variable
To use a categorical variables like `Winery`, we can simply pass it to the formula, and `smf.ols` will handle the variable.

In [73]:
modTwo = smf.ols(formula='LogAuction ~ Winery + WinterRain + HarvestRain + GrowTemp + Age',
                 data=wine_two_train).fit()
print(modTwo.summary())

                            OLS Regression Results                            
Dep. Variable:             LogAuction   R-squared:                       0.860
Model:                            OLS   Adj. R-squared:                  0.836
Method:                 Least Squares   F-statistic:                     35.55
Date:                Sun, 11 Oct 2020   Prob (F-statistic):           1.62e-11
Time:                        17:02:15   Log-Likelihood:                -13.178
No. Observations:                  35   AIC:                             38.36
Df Residuals:                      29   BIC:                             47.69
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

### More Wineries
Now let's expand to the complete set of values that `Winery` can take:

In [74]:
modNew = smf.ols(formula='LogAuction ~ Winery + WinterRain + HarvestRain + GrowTemp + Age',
                 data=wine_new_train).fit()
print(modNew.summary())

                            OLS Regression Results                            
Dep. Variable:             LogAuction   R-squared:                       0.851
Model:                            OLS   Adj. R-squared:                  0.834
Method:                 Least Squares   F-statistic:                     52.68
Date:                Sun, 11 Oct 2020   Prob (F-statistic):           1.73e-27
Time:                        17:02:20   Log-Likelihood:                -43.659
No. Observations:                  83   AIC:                             105.3
Df Residuals:                      74   BIC:                             127.1
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

### Evaluate our final model

In [79]:
# compute out-of-sample R squared
cols = ['Winery', 'WinterRain', 'HarvestRain', 'GrowTemp', 'Age']

X_test = wine_new_test[cols]
y_test = wine_new_test['LogAuction']

X_test = pd.get_dummies(X_test, columns=['Winery'])
X_test = sm.add_constant(X_test)

y_pred = modNew.predict(X_test)
SSE = np.sum((y_test - y_pred)**2)
SST = np.sum((y_test - np.mean(y_train))**2)
OSR2 = 1 - SSE/SST
print(OSR2)

PatsyError: predict requires that you use a DataFrame when predicting from a model
that was created using the formula api.

The original error message returned by patsy is:
Error evaluating factor: NameError: name 'Winery' is not defined
    LogAuction ~ Winery + WinterRain + HarvestRain + GrowTemp + Age
                 ^^^^^^

# -----------------------------------------------------------------------------------------

# Statsmodels (using matrices)

In [ ]:
# # calculate Variance Inflation Factor for each explanatory variable
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# # exclude the column related to the constant
# vif = [variance_inflation_factor(X_train.values, i) for i in range(1, X_train.shape[1])] 
# print('\nVIF:')
# print(pd.Series(np.array(vif), index=X_train.columns[1:]))

In [ ]:
# # remove FrancePop
# X_train.drop(columns=['FrancePop'], axis=1, inplace=True)

# model2 = sm.OLS(y_train, X_train).fit()
# print(model2.summary())

# vif = [variance_inflation_factor(X_train.values, i) for i in range(1, X_train.shape[1])] 
# print('\nVIF:')
# print(pd.Series(np.array(vif), index=X_train.columns[1:]))

In [ ]:
# # remove USAlcConsump
# X_train.drop(columns=['USAlcConsump'], axis=1, inplace=True)

# model3 = sm.OLS(y_train, X_train).fit()
# print(model3.summary())

# vif = [variance_inflation_factor(X_train.values, i) for i in range(1, X_train.shape[1])] 
# print('\nVIF:')
# print(pd.Series(np.array(vif), index=X_train.columns[1:]))

In [ ]:
# # remove HarvestTemp
# X_train.drop(columns=['HarvestTemp'], axis=1, inplace=True)

# model4 = sm.OLS(y_train, X_train).fit()
# print(model4.summary())

# vif = [variance_inflation_factor(X_train.values, i) for i in range(1, X_train.shape[1])] 
# print('\nVIF:')
# print(pd.Series(np.array(vif), index=X_train.columns[1:]))

In [66]:
# ### Dummy/One-hot Encoding
# To use a categorical variables like `Winery`, we need to perform an encoding of the string. 
# That is, given the discrete set of possible values that the string can take, we use dummy encoding (also know as one-hot enconding), to disaggregate the variable into distinct binary variables:
# https://stackoverflow.com/questions/55738056/using-categorical-variables-in-statsmodels-ols-class

In [ ]:
# cols = ['Winery', 'WinterRain', 'HarvestRain', 'GrowTemp', 'Age']
# X_train = wine_two_train[cols]
# y_train = wine_two_train['LogAuction']

# X_train = pd.get_dummies(X_train, columns=['Winery'])
# X_train = sm.add_constant(X_train)

# modTwo = sm.OLS(y_train, X_train).fit()
# print(modTwo.summary())

In [ ]:
# ### More Wineries
# Now let's expand to the complete set of values that `Winery` can take:

In [ ]:
# cols = ['Winery', 'WinterRain', 'HarvestRain', 'GrowTemp', 'Age']
# X_train = wine_new_train[cols]
# y_train = wine_new_train['LogAuction']

# X_train = pd.get_dummies(X_train, columns=['Winery'])
# X_train = sm.add_constant(X_train)

# modNew = sm.OLS(y_train, X_train).fit()
# print(modNew.summary())

In [ ]:
# ### Evaluate our final model

In [ ]:
# # compute out-of-sample R squared
# X_test = wine_new_test[cols]
# y_test = wine_new_test['LogAuction']

# X_test = pd.get_dummies(X_test, columns=['Winery'])
# X_test = sm.add_constant(X_test)

# y_pred = modNew.predict(X_test)
# SSE = np.sum((y_test - y_pred)**2)
# SST = np.sum((y_test - np.mean(y_train))**2)
# OSR2 = 1 - SSE/SST
# print(OSR2)

# -----------------------------------------------------------------------------------------

# Scikit Learn version (90% complete)

In [ ]:
# # train the model
# from sklearn.linear_model import LinearRegression

# # choose the features to be used
# cols = ['WinterRain', 'HarvestRain', 'GrowTemp', 'HarvestTemp', 'Age', 'FrancePop', 'USAlcConsump']
# X_train = wine_train[cols]
# y_train = wine_train['LogAuctionIndex']

# model1 = LinearRegression().fit(X_train, y_train)
# R_squared = model1.score(X_train, y_train)

# print('R_squared:', R_squared)

# intercept = model1.intercept_
# coefficients = model1.coef_

# print('Intercept:', intercept)
# print('Coefficients:', coefficients)

# # compute out-of-sample R_squared
# X_test = wine_test[cols]
# y_test = wine_test['LogAuctionIndex']

# wine_predictions = model1.predict(X_test)
# OSR_squared = model1.score(X_test, y_test)
# print(OSR_squared)

# # Compute Variance Inflation Factor
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from patsy import dmatrices

# # find design matrix for linear regression model using 'rating' as response variable
# y, X = dmatrices('LogAuctionIndex ~ WinterRain + HarvestRain + GrowTemp + HarvestTemp + Age + FrancePop + USAlcConsump', data=wine_train, return_type='dataframe')

# #calculate VIF for each explanatory variable
# vif = pd.DataFrame()
# vif['Variable'] = cols
# vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
# print(vif)

# # A better model...
# # Remove FrancePop
# cols = ['WinterRain', 'HarvestRain', 'GrowTemp', 'HarvestTemp', 'Age', 'USAlcConsump']
# X_train = wine_train[cols]

# model2 = LinearRegression().fit(X_train, y_train)
# R_squared = model2.score(X_train, y_train)

# print('R_squared:', R_squared)
# print('Intercept:', model2.intercept_)
# print('Coefficients:', model2.coef_)

# #calculate VIF for each explanatory variable
# vif = pd.DataFrame()
# vif['Variable'] = cols
# vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
# print(vif)

# # Remove USAlcConsump
# cols = ['WinterRain', 'HarvestRain', 'GrowTemp', 'HarvestTemp', 'Age']
# X_train = wine_train[cols]

# model3 = LinearRegression().fit(X_train, y_train)
# R_squared = model3.score(X_train, y_train)

# print('R_squared:', R_squared)
# print('Intercept:', model3.intercept_)
# print('Coefficients:', model3.coef_)

# vif = pd.DataFrame()
# vif['Variable'] = cols
# vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
# print(vif)

# # Remove HarvestTemp
# cols = ['WinterRain', 'HarvestRain', 'GrowTemp', 'Age']
# X_train = wine_train[cols]

# model4 = LinearRegression().fit(X_train, y_train)
# R_squared = model4.score(X_train, y_train)

# print('R_squared:', R_squared)
# print('Intercept:', model4.intercept_)
# print('Coefficients:', model4.coef_)

# vif = pd.DataFrame()
# vif['Variable'] = cols
# vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
# print(vif)

# # ## produce a model that outputs the effect of an increase of log(1) of USAlcConsump controlling for Age
# # # whats' the coefficient associated with 



# wine_train['GrowTemp*Age'] = wine_train['GrowTemp']*wine_train['Age']
# cols = ['WinterRain', 'HarvestRain', 'GrowTemp', 'Age', 'GrowTemp*Age']
# X_train = wine_train[cols]

# model5 = LinearRegression().fit(X_train, y_train)
# R_squared = model5.score(X_train, y_train)

# print('R_squared:', R_squared)
# print('Intercept:', model5.intercept_)
# print('Coefficients:', model5.coef_)

# # # Categorical Variables
# wine_new = pd.read_csv("wine_disagg.csv")
# wine_new_train = wine_new[wine_new['Year'] <= 1985]
# wine_new_test = wine_new[wine_new['Year'] > 1985]

# wine_new.info()
# wine_new.head()

# # # Regression use new data
# cols = ['WinterRain', 'HarvestRain', 'GrowTemp', 'Age']
# y_train = wine_new_train['LogAuction']
# X_train = wine_new_train[cols]

# modOld = LinearRegression().fit(X_train, y_train)
# R_squared = modOld.score(X_train, y_train)

# print('R_squared:', R_squared)
# print('Intercept:', modOld.intercept_)
# print('Coefficients:', modOld.coef_)

# # # Two wineries
# wine_two = wine_new[(wine_new['Winery'] == 'Cheval Blanc') | (wine_new['Winery'] == 'Cos d\'Estournel')]
# # ggplot(wine.two, aes(x=Year, y=LogAuction, color=Winery)) + geom_line() +
# #   xlab("Vintage Year") + ylab("log(2015 Auction Price)")

# wine_two_train = wine_two[wine_two['Year'] <= 1985]
# wine_two_test = wine_two[wine_two['Year'] > 1985]
# wine_two_train.tail()

# cols = ['WinterRain', 'HarvestRain', 'GrowTemp', 'Age']
# X_train = wine_two_train[cols]
# y_train = wine_two_train['LogAuction']

# modTwo = LinearRegression().fit(X_train, y_train)
# R_squared = modTwo.score(X_train, y_train)

# print('R_squared:', R_squared)
# print('Intercept:', modTwo.intercept_)
# print('Coefficients:', modTwo.coef_)
# # modTwo <- lm(LogAuction ~ Winery + WinterRain + HarvestRain + GrowTemp + Age, 
# #              data = wine.two.train)
# # summary(modTwo)

# cols = ['Winery', 'WinterRain', 'HarvestRain', 'GrowTemp', 'Age']
# X_train = wine_two_train[cols]
# y_train = wine_two_train['LogAuction']

# modNew = LinearRegression().fit(X_train, y_train)
# R_squared = modNew.score(X_train, y_train)

# print('R_squared:', R_squared)
# print('Intercept:', modTwo.intercept_)
# print('Coefficients:', modTwo.coef_)

# # # compute OSR^2
# X_test = wine_new_train[cols]
# y_test = wine_new_test['LogAuction']

# wine_predictions = modNew.predict(X_test)
# OSR_squared = modNew.score(X_test, y_test)
# print(OSR_squared)